In [1]:
import numpy as np
from qiskit.circuit import ParameterVector
from qiskit.quantum_info import SparsePauliOp
from qiskit.primitives import Estimator,Sampler

from squlearn.util.optree.optree_derivative import derivative
from squlearn.util.optree.optree_evaluate import build_operator_list,evaluate_estimator,evaluate_sampler
from squlearn.util.optree import *

from squlearn.expectation_operator import SummedPaulis
from squlearn.feature_map import ChebRx

In [2]:
op = SummedPaulis(2)
pop=ParameterVector('pop', op.num_parameters)
e = op.get_pauli_new(pop)
print(e)

SparsePauliOp(['II', 'IZ', 'ZI'],
              coeffs=[ParameterExpression(1.0*pop[0]), ParameterExpression(1.0*pop[1]),
 ParameterExpression(1.0*pop[2])])


In [3]:
op_grad = derivative(e,pop)
print(op_grad)

[1.0*SparsePauliOp(['II'],
              coeffs=[1.+0.j]), 1.0*SparsePauliOp(['IZ'],
              coeffs=[1.+0.j]), 1.0*SparsePauliOp(['ZI'],
              coeffs=[1.+0.j])]


In [4]:
fm = ChebRx(2,1)
x = ParameterVector("x",fm.num_features)
p = ParameterVector("p",fm.num_parameters)
fm_circ = fm.get_circuit(x,p)

fm_grad = derivative(fm_circ,p)
print(fm_grad)

[1.0*(0.5*acos(x[0])*
     ┌───────────────────────────┐┌──────────┐     
q_0: ┤ Rx(p[0]*acos(x[0]) + π/2) ├┤ Rx(p[2]) ├──■──
     └──┬─────────────────────┬──┘├──────────┤┌─┴─┐
q_1: ───┤ Rx(p[1]*acos(x[0])) ├───┤ Rx(p[3]) ├┤ X ├
        └─────────────────────┘   └──────────┘└───┘
 + -0.5*acos(x[0])*
     ┌───────────────────────────┐┌──────────┐     
q_0: ┤ Rx(p[0]*acos(x[0]) - π/2) ├┤ Rx(p[2]) ├──■──
     └──┬─────────────────────┬──┘├──────────┤┌─┴─┐
q_1: ───┤ Rx(p[1]*acos(x[0])) ├───┤ Rx(p[3]) ├┤ X ├
        └─────────────────────┘   └──────────┘└───┘
), 1.0*(0.5*acos(x[0])*
        ┌─────────────────────┐   ┌──────────┐     
q_0: ───┤ Rx(p[0]*acos(x[0])) ├───┤ Rx(p[2]) ├──■──
     ┌──┴─────────────────────┴──┐├──────────┤┌─┴─┐
q_1: ┤ Rx(p[1]*acos(x[0]) + π/2) ├┤ Rx(p[3]) ├┤ X ├
     └───────────────────────────┘└──────────┘└───┘
 + -0.5*acos(x[0])*
        ┌─────────────────────┐   ┌──────────┐     
q_0: ───┤ Rx(p[0]*acos(x[0])) ├───┤ Rx(p[2]) ├──■──
     ┌──┴─────────────────────

In [5]:
#dictionary1={pop[0]: 0.5,pop[1]:0.75, pop[2]:1.0, x[0]:-0.5,p[0]:10.0,p[1]:20.0,p[2]:30.0,p[3]:40.0}

#estim = Estimator()

#evaluate_estimator(fm_grad,op_grad,dictionary1,estim)

In [6]:
dictionary_c1 = {x[0]:-0.5,p[0]:10.0,p[1]:20.0,p[2]:30.0,p[3]:40.0}
dictionary_c2 = {x[0]: 0.1,p[0]:10.0,p[1]:20.0,p[2]:30.0,p[3]:40.0}

dictionary_op = {pop[0]: 0.5,pop[1]:0.75, pop[2]:1.0}

estim = Estimator()

evaluate_estimator(fm_grad,op_grad,[dictionary_c1,dictionary_c2],dictionary_op,estim)

run time 0.037011146545410156
estimator_result [ 1.         -0.62760149 -0.61426869  1.          0.62760149  0.61426869
  1.          0.77853476 -0.15962199  1.          0.77853476  0.15962199
  1.         -0.62760149 -0.61426869  1.          0.62760149  0.61426869
  1.          0.77853476 -0.15962199  1.          0.77853476  0.15962199
  1.         -0.66238055 -0.63327598  1.          0.66238055  0.63327598
  1.          0.74916754 -0.21963265  1.          0.74916754  0.21963265
  1.         -0.66238055 -0.63327598  1.          0.66238055  0.63327598
  1.          0.74916754 -0.21963265  1.          0.74916754  0.21963265]
evaluation_tree [1.0*[1.0*(1.0471975511965979*[1.0*0, 1.0*1, 1.0*2] + -1.0471975511965979*[1.0*3, 1.0*4, 1.0*5]), 1.0*(1.0471975511965979*[1.0*6, 1.0*7, 1.0*8] + -1.0471975511965979*[1.0*9, 1.0*10, 1.0*11]), 1.0*(0.5*[1.0*12, 1.0*13, 1.0*14] + -0.5*[1.0*15, 1.0*16, 1.0*17]), 1.0*(0.5*[1.0*18, 1.0*19, 1.0*20] + -0.5*[1.0*21, 1.0*22, 1.0*23])], 1.0*[1.0*(0.73531445281

array([[[ 0.00000000e+00, -1.31444548e+00, -1.28652134e+00],
        [ 2.22044605e-16,  3.33066907e-16, -3.34311511e-01],
        [ 5.55111512e-17, -6.27601486e-01, -6.14268693e-01],
        [ 5.55111512e-17,  1.11022302e-16, -1.59621989e-01]],

       [[-1.11022302e-16, -9.74115981e-01, -9.31313963e-01],
        [ 0.00000000e+00,  0.00000000e+00, -3.22998123e-01],
        [ 5.55111512e-17, -6.62380548e-01, -6.33275981e-01],
        [ 0.00000000e+00, -5.55111512e-17, -2.19632649e-01]]])

In [7]:
dictionary_c1 = {x[0]:-0.5,p[0]:10.0,p[1]:20.0,p[2]:30.0,p[3]:40.0}
dictionary_c2 = {x[0]: 0.1,p[0]:10.0,p[1]:20.0,p[2]:30.0,p[3]:40.0}

dictionary_op1 = {pop[0]: 0.5,pop[1]:0.75, pop[2]:1.0}
dictionary_op2 = {pop[0]: 0.5,pop[1]:0.75, pop[2]:1.0}

estim = Estimator()

evaluate_estimator(fm_grad,op_grad,[dictionary_c1,dictionary_c2],[dictionary_op1,dictionary_op2],estim,dictionaries_combined=True)

run time 0.03999900817871094
estimator_result [ 1.         -0.62760149 -0.61426869  1.          0.62760149  0.61426869
  1.          0.77853476 -0.15962199  1.          0.77853476  0.15962199
  1.         -0.62760149 -0.61426869  1.          0.62760149  0.61426869
  1.          0.77853476 -0.15962199  1.          0.77853476  0.15962199
  1.         -0.66238055 -0.63327598  1.          0.66238055  0.63327598
  1.          0.74916754 -0.21963265  1.          0.74916754  0.21963265
  1.         -0.66238055 -0.63327598  1.          0.66238055  0.63327598
  1.          0.74916754 -0.21963265  1.          0.74916754  0.21963265]
evaluation_tree [1.0*[1.0*(1.0471975511965979*[1.0*0, 1.0*1, 1.0*2] + -1.0471975511965979*[1.0*3, 1.0*4, 1.0*5]), 1.0*(1.0471975511965979*[1.0*6, 1.0*7, 1.0*8] + -1.0471975511965979*[1.0*9, 1.0*10, 1.0*11]), 1.0*(0.5*[1.0*12, 1.0*13, 1.0*14] + -0.5*[1.0*15, 1.0*16, 1.0*17]), 1.0*(0.5*[1.0*18, 1.0*19, 1.0*20] + -0.5*[1.0*21, 1.0*22, 1.0*23])], 1.0*[1.0*(0.735314452816

array([[[ 0.00000000e+00, -1.31444548e+00, -1.28652134e+00],
        [ 2.22044605e-16,  3.33066907e-16, -3.34311511e-01],
        [ 5.55111512e-17, -6.27601486e-01, -6.14268693e-01],
        [ 5.55111512e-17,  1.11022302e-16, -1.59621989e-01]],

       [[-1.11022302e-16, -9.74115981e-01, -9.31313963e-01],
        [ 0.00000000e+00,  0.00000000e+00, -3.22998123e-01],
        [ 5.55111512e-17, -6.62380548e-01, -6.33275981e-01],
        [ 0.00000000e+00, -5.55111512e-17, -2.19632649e-01]]])

In [8]:
from squlearn.util import Executor
from squlearn.qnn.qnn import QNN

executor = Executor(Estimator())
qnn = QNN(fm,op,executor)

res = qnn.evaluate_from_string("dfdopdp",x=[-0.5,0.1],param=[10,20,30,40],param_op=[0.5,0.75,1.0])



print(res[0].transpose())
print(res[1].transpose())

[[ 0.00000000e+00 -1.31444548e+00 -1.28652134e+00]
 [ 0.00000000e+00  2.22044605e-16 -3.34311511e-01]
 [ 0.00000000e+00 -6.27601486e-01 -6.14268693e-01]
 [ 5.55111512e-17  5.55111512e-17 -1.59621989e-01]]
[[ 1.11022302e-16 -9.74115981e-01 -9.31313963e-01]
 [ 0.00000000e+00  1.11022302e-16 -3.22998123e-01]
 [ 0.00000000e+00 -6.62380548e-01 -6.33275981e-01]
 [ 1.11022302e-16  1.66533454e-16 -2.19632649e-01]]


In [9]:
print(fm_grad)

[1.0*(0.5*acos(x[0])*
     ┌───────────────────────────┐┌──────────┐     
q_0: ┤ Rx(p[0]*acos(x[0]) + π/2) ├┤ Rx(p[2]) ├──■──
     └──┬─────────────────────┬──┘├──────────┤┌─┴─┐
q_1: ───┤ Rx(p[1]*acos(x[0])) ├───┤ Rx(p[3]) ├┤ X ├
        └─────────────────────┘   └──────────┘└───┘
 + -0.5*acos(x[0])*
     ┌───────────────────────────┐┌──────────┐     
q_0: ┤ Rx(p[0]*acos(x[0]) - π/2) ├┤ Rx(p[2]) ├──■──
     └──┬─────────────────────┬──┘├──────────┤┌─┴─┐
q_1: ───┤ Rx(p[1]*acos(x[0])) ├───┤ Rx(p[3]) ├┤ X ├
        └─────────────────────┘   └──────────┘└───┘
), 1.0*(0.5*acos(x[0])*
        ┌─────────────────────┐   ┌──────────┐     
q_0: ───┤ Rx(p[0]*acos(x[0])) ├───┤ Rx(p[2]) ├──■──
     ┌──┴─────────────────────┴──┐├──────────┤┌─┴─┐
q_1: ┤ Rx(p[1]*acos(x[0]) + π/2) ├┤ Rx(p[3]) ├┤ X ├
     └───────────────────────────┘└──────────┘└───┘
 + -0.5*acos(x[0])*
        ┌─────────────────────┐   ┌──────────┐     
q_0: ───┤ Rx(p[0]*acos(x[0])) ├───┤ Rx(p[2]) ├──■──
     ┌──┴─────────────────────

In [10]:
sampler = Sampler()

evaluate_sampler(fm_grad,op_grad,[dictionary_c1,dictionary_c2],dictionary_op1,sampler)

sampler_result SamplerResult(quasi_dists=[{0: 0.1842214443332138, 1: 0.0086442091953682, 2: 0.0019778125569535, 3: 0.8051565339144646}, {0: 0.8051565339144632, 1: 0.0019778125569535, 2: 0.0086442091953682, 3: 0.1842214443332151}, {0: 0.3534710121865028, 1: 0.0667179935059654, 2: 0.5357963684450073, 3: 0.0440146258625241}, {0: 0.5357963684450058, 1: 0.0440146258625243, 2: 0.3534710121865043, 3: 0.0667179935059653}, {0: 0.1842214443332138, 1: 0.0086442091953682, 2: 0.0019778125569535, 3: 0.8051565339144644}, {0: 0.8051565339144632, 1: 0.0019778125569535, 2: 0.0086442091953682, 3: 0.1842214443332151}, {0: 0.3534710121865028, 1: 0.0667179935059655, 2: 0.5357963684450073, 3: 0.0440146258625242}, {0: 0.5357963684450058, 1: 0.0440146258625243, 2: 0.3534710121865043, 3: 0.0667179935059653}], metadata=[{}, {}, {}, {}, {}, {}, {}, {}])
